**INFORMAÇÕES**



> **Notebook referente a segunda questão da segunda prova da disciplina Introdução a Inteligência Artifical (UFPB/2020.1), ministrada pela professora Dra. Thaís Gaudencio do Rêgo.**

**Discente**: Thomás Augusto Gouveia Chaves

**Matrícula**: 20180002671

*Enunciado: Utilizando a base de dados, utilize dois algoritmos não supervisionados, podendo ser K-means, Hierárquico ou outros de sua escolha. Ambos os modelos devem ser analisados através da métrica silhueta ou outra métrica de sua escolha. Os resultados devem ser discutidos.*


**BIBLIOTECAS UTILIZADAS**

In [ ]:
import pandas as pd
import numpy as np
from time import time
from sklearn import preprocessing
from sklearn import metrics
from sklearn.cluster import KMeans, Birch, AgglomerativeClustering
from sklearn.decomposition import PCA

**CARREGAMENTO DA BASE DE DADOS**

Base de dados carregada a partir da URL direta.

In [ ]:
data_url = "https://drive.google.com/u/0/uc?id=16DZggHr5lShbHj0ZscKNiYvMVm8mTrX_&export=download"

In [ ]:
dataset = pd.read_csv(data_url)
dataset.head()

,Unnamed: 0,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,OD280/OD315_of_diluted_wines,Proline
0,0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


**PRÉ PROCESSAMENTO DA BASE DE DADOS**


**Unamed: 0**
Remoção da coluna 'Unamed: 0', irrelevante para a construção do método, tendo em vista que os valores da coluna são referentes aos números das instâncias.

In [ ]:
dataset = dataset.drop('Unnamed: 0', axis=1)
dataset.shape

(178, 13)


**Valores faltosos ou duplicados** Remoção de instâncias com dados faltosos ou duplicados.

**Comentário:** A base de dados não contém nenhuma instância com estes problemas. Mas, para uma melhor portabilidade do código, deixei implementado esta parte.

In [ ]:
dataset = dataset.dropna()
dataset = dataset.drop_duplicates()
dataset.shape

(178, 13)

**ANÁLISE DE CORRELAÇÃO ENTRE OS ATRIBUTOS**

In [ ]:
corr = dataset.corr()
corr.style.background_gradient(cmap='coolwarm')

,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Total_phenols,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,OD280/OD315_of_diluted_wines,Proline
Alcohol,1.000000,0.094397,0.211545,-0.310235,0.270798,0.289101,0.236815,-0.155929,0.136698,0.546364,-0.071747,0.072343,0.643720
Malic_acid,0.094397,1.000000,0.164045,0.288500,-0.054575,-0.335167,-0.411007,0.292977,-0.220746,0.248985,-0.561296,-0.368710,-0.192011
Ash,0.211545,0.164045,1.000000,0.443367,0.286587,0.128980,0.115077,0.186230,0.009652,0.258887,-0.074667,0.003911,0.223626
Alcalinity_of_ash,-0.310235,0.288500,0.443367,1.000000,-0.083333,-0.321113,-0.351370,0.361922,-0.197327,0.018732,-0.273955,-0.276769,-0.440597
Magnesium,0.270798,-0.054575,0.286587,-0.083333,1.000000,0.214401,0.195784,-0.256294,0.236441,0.199950,0.055398,0.066004,0.393351
Total_phenols,0.289101,-0.335167,0.128980,-0.321113,0.214401,1.000000,0.864564,-0.449935,0.612413,-0.055136,0.433681,0.699949,0.498115
Flavanoids,0.236815,-0.411007,0.115077,-0.351370,0.195784,0.864564,1.000000,-0.537900,0.652692,-0.172379,0.543479,0.787194,0.494193
Nonflavanoid_phenols,-0.155929,0.292977,0.186230,0.361922,-0.256294,-0.449935,-0.537900,1.000000,-0.365845,0.139057,-0.262640,-0.503270,-0.311385
Proanthocyanins,0.136698,-0.220746,0.009652,-0.197327,0.236441,0.612413,0.652692,-0.365845,1.000000,-0.025250,0.295544,0.519067,0.330417
Color_intensity,0.546364,0.248985,0.258887,0.018732,0.199950,-0.055136,-0.172379,0.139057,-0.025250,1.000000,-0.521813,-0.428815,0.316100



**Correlações de atributos > 75%**

Flavanoids e Total_phenols: 86,4564%

OD280/OD315_of_diluted_wines e Flavanoids: 78,7194%

Há dois pares de atributos com uma alta correlação. O atributo Flavanoids faz parte destes dois pares. Portanto, para um bom aproveitamento desta correlação, podemos remover os outros dois atributos que possuem uma correlação maior 75%, Total_phenols e OD280/OD315_of_diluted_wines.

**REMOÇÃO DE ATRIBUTOS COM ALTA CORRELAÇÃO**

In [ ]:
dataset = dataset.drop('Total_phenols', axis=1)
dataset = dataset.drop('OD280/OD315_of_diluted_wines', axis=1)


**Nova análise da correlação**

In [ ]:
corr = dataset.corr()
corr.style.background_gradient(cmap='coolwarm')

,Alcohol,Malic_acid,Ash,Alcalinity_of_ash,Magnesium,Flavanoids,Nonflavanoid_phenols,Proanthocyanins,Color_intensity,Hue,Proline
Alcohol,1.000000,0.094397,0.211545,-0.310235,0.270798,0.236815,-0.155929,0.136698,0.546364,-0.071747,0.643720
Malic_acid,0.094397,1.000000,0.164045,0.288500,-0.054575,-0.411007,0.292977,-0.220746,0.248985,-0.561296,-0.192011
Ash,0.211545,0.164045,1.000000,0.443367,0.286587,0.115077,0.186230,0.009652,0.258887,-0.074667,0.223626
Alcalinity_of_ash,-0.310235,0.288500,0.443367,1.000000,-0.083333,-0.351370,0.361922,-0.197327,0.018732,-0.273955,-0.440597
Magnesium,0.270798,-0.054575,0.286587,-0.083333,1.000000,0.195784,-0.256294,0.236441,0.199950,0.055398,0.393351
Flavanoids,0.236815,-0.411007,0.115077,-0.351370,0.195784,1.000000,-0.537900,0.652692,-0.172379,0.543479,0.494193
Nonflavanoid_phenols,-0.155929,0.292977,0.186230,0.361922,-0.256294,-0.537900,1.000000,-0.365845,0.139057,-0.262640,-0.311385
Proanthocyanins,0.136698,-0.220746,0.009652,-0.197327,0.236441,0.652692,-0.365845,1.000000,-0.025250,0.295544,0.330417
Color_intensity,0.546364,0.248985,0.258887,0.018732,0.199950,-0.172379,0.139057,-0.025250,1.000000,-0.521813,0.316100
Hue,-0.071747,-0.561296,-0.074667,-0.273955,0.055398,0.543479,-0.262640,0.295544,-0.521813,1.000000,0.236183


**Comentário:** Após a remoção dos atributos correlacionados a base de dados foi reduzida em dois atributos, melhorando assim o processamento.

**REDUÇÃO DE ATRIBUTOS COM PRINCIPAL COMPONENT ANALISYS (PCA)**

Aplicação do método PCA para a redução de atributos. Foram reduzidos de 9 atributos (11 para 2).

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)
X_pca = pd.DataFrame(data = principalComponents, columns = ['0', '1'])

**NORMALIZAÇÃO DA BASE DE DADOS**

Normalização da base de dados com o método StandardScaler.

In [ ]:
x_norm = dataset.values
scaler = preprocessing.StandardScaler()
x_scaled = scaler.fit_transform(x_norm)
X_normalized = pd.DataFrame(x_scaled)

In [ ]:
x_pca_n = X_pca.values
x_pca_scaled = scaler.fit_transform(x_pca_n)
X_pca_normalized = pd.DataFrame(x_pca_scaled)

**FUNÇÃO PARA ANÁLISE DOS MÉTODOS**

Esta função encapsula as chamadas dos método de aprendizagem e avaliação (silhueta). Também imprime o tempo de processamento do método .

In [ ]:
def bench(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    clusters = len(np.unique(estimator.labels_))
    print('Name\t\tClusters\tTime\tSilhouette\n%-9s\t\t%i\t%.4fs\t%.3f'
          % (name, clusters, (time() - t0), metrics.silhouette_score(data, estimator.labels_,metric='euclidean')))


**APLICAÇÃO E ANÁLISE DE MÉTODOS DE AGRUPAMENTO**

Nesta seção, são testados 3 métodos de agrupamento: Aglomerativo(hierárquico), K-means e Birch. Os testes foram feitos para a base normalizada, não normalizada, com redução de atributos e com redução de atributos normalizada.
Foram testados k=2, k=3 e k=4, sendo k o número de clusters.

In [ ]:
X = dataset


**Método hierárquico**

In [ ]:
print("\nClusters: 2 (non-normalized data):")
bench(AgglomerativeClustering(n_clusters=2), "Hierarchical", X)
print("\nClusters: 2 (normalized data):")
print(bench(AgglomerativeClustering(n_clusters=2), "Hierarchical", X_normalized))

print("\nClusters: 3 (non-normalized data):")
bench(AgglomerativeClustering(n_clusters=3), "Hierarchical", X)
print("\nClusters: 3 (normalized data):")
print(bench(AgglomerativeClustering(n_clusters=3), "Hierarchical", X_normalized))

print("\nClusters: 4 (non-normalized data):")
bench(AgglomerativeClustering(n_clusters=4), "Hierarchical", X)
print("\nClusters: 4 (normalized data):")
print(bench(AgglomerativeClustering(n_clusters=4), "Hierarchical", X_normalized))

#PCA
print("\nData with PCA:")
print("\nClusters: 2 (PCA non-normalized data):")
bench(AgglomerativeClustering(n_clusters=2), "Hierarchical", X_pca)
print("\nClusters: 2 (PCA normalized data):")
print(bench(AgglomerativeClustering(n_clusters=2), "Hierarchical", X_pca_normalized))

print("\nClusters: 3 (PCA non-normalized data):")
bench(AgglomerativeClustering(n_clusters=3), "Hierarchical", X_pca)
print("\nClusters: 3 (PCA normalized data):")
print(bench(AgglomerativeClustering(n_clusters=3), "Hierarchical", X_pca_normalized))

print("\nClusters: 4 (PCA non-normalized data):")
bench(AgglomerativeClustering(n_clusters=4), "Hierarchical", X_pca)
print("\nClusters: 4 (PCA normalized data):")
print(bench(AgglomerativeClustering(n_clusters=4), "Hierarchical", X_pca_normalized))


Clusters: 2 (non-normalized data):
Name		Clusters	Time	Silhouette
Hierarchical		2	0.0028s	0.659

Clusters: 2 (normalized data):
Name		Clusters	Time	Silhouette
Hierarchical		2	0.0032s	0.236
None

Clusters: 3 (non-normalized data):
Name		Clusters	Time	Silhouette
Hierarchical		3	0.0032s	0.565

Clusters: 3 (normalized data):
Name		Clusters	Time	Silhouette
Hierarchical		3	0.0028s	0.257
None

Clusters: 4 (non-normalized data):
Name		Clusters	Time	Silhouette
Hierarchical		4	0.0035s	0.561

Clusters: 4 (normalized data):
Name		Clusters	Time	Silhouette
Hierarchical		4	0.0032s	0.206
None

Data with PCA:

Clusters: 2 (PCA non-normalized data):
Name		Clusters	Time	Silhouette
Hierarchical		2	0.0037s	0.659

Clusters: 2 (PCA normalized data):
Name		Clusters	Time	Silhouette
Hierarchical		2	0.0028s	0.373
None

Clusters: 3 (PCA non-normalized data):
Name		Clusters	Time	Silhouette
Hierarchical		3	0.0030s	0.566

Clusters: 3 (PCA normalized data):
Name		Clusters	Time	Silhouette
Hierarchical		3	0.0029s	0.42


**Método K-means**

In [ ]:
print("\nClusters: 2 (non-normalized data):")
bench(KMeans(n_clusters=2, random_state=0), "K-means", X)
print("\nClusters: 2 (normalized data):")
bench(KMeans(n_clusters=2, random_state=0), "K-means", X_normalized)

print("\nClusters: 3 (non-normalized data):")
bench(KMeans(n_clusters=3, random_state=0), "K-means", X)
print("\nClusters: 3 (normalized data):")
bench(KMeans(n_clusters=3, random_state=0), "K-means", X_normalized)

print("\nClusters: 4 (non-normalized data):")
bench(KMeans(n_clusters=4, random_state=0), "K-means", X)
print("\nClusters: 4 (normalized data):")
bench(KMeans(n_clusters=4, random_state=0), "K-means", X_normalized)

#PCA
print("\nData with PCA:")
print("\nClusters: 2 (PCA non-normalized data):")
bench(KMeans(n_clusters=2, random_state=0), "K-means", X_pca)
print("\nClusters: 2 (PCA normalized data):")
bench(KMeans(n_clusters=2, random_state=0), "K-means", X_pca_normalized)

print("\nClusters: 3 (PCA non-normalized data):")
bench(KMeans(n_clusters=3, random_state=0), "K-means", X_pca)
print("\nClusters: 3 (PCA normalized data):")
bench(KMeans(n_clusters=3, random_state=0), "K-means", X_pca_normalized)

print("\nClusters: 4 (PCA non-normalized data):")
bench(KMeans(n_clusters=4, random_state=0), "K-means", X_pca)
print("\nClusters: 4 (PCA normalized data):")
bench(KMeans(n_clusters=4, random_state=0), "K-means", X_pca_normalized)


Clusters: 2 (non-normalized data):
Name		Clusters	Time	Silhouette
K-means  		2	0.0363s	0.657

Clusters: 2 (normalized data):
Name		Clusters	Time	Silhouette
K-means  		2	0.0311s	0.236

Clusters: 3 (non-normalized data):
Name		Clusters	Time	Silhouette
K-means  		3	0.0454s	0.571

Clusters: 3 (normalized data):
Name		Clusters	Time	Silhouette
K-means  		3	0.0381s	0.281

Clusters: 4 (non-normalized data):
Name		Clusters	Time	Silhouette
K-means  		4	0.0428s	0.561

Clusters: 4 (normalized data):
Name		Clusters	Time	Silhouette
K-means  		4	0.0483s	0.272

Data with PCA:

Clusters: 2 (PCA non-normalized data):
Name		Clusters	Time	Silhouette
K-means  		2	0.0310s	0.657

Clusters: 2 (PCA normalized data):
Name		Clusters	Time	Silhouette
K-means  		2	0.0323s	0.413

Clusters: 3 (PCA non-normalized data):
Name		Clusters	Time	Silhouette
K-means  		3	0.0431s	0.572

Clusters: 3 (PCA normalized data):
Name		Clusters	Time	Silhouette
K-means  		3	0.0420s	0.475

Clusters: 4 (PCA non-normalized data):
Name		Cl


**Método Birch**

In [ ]:
print("\nClusters: 2 (non-normalized data):")
bench(Birch(n_clusters=2), "Birch", X)
print("\nClusters: 2 (normalized data):")
bench(Birch(n_clusters=2), "Birch", X_normalized)

print("\nClusters: 3 (non-normalized data):")
bench(Birch(n_clusters=3), "Birch", X)
print("\nClusters: 3 (normalized data):")
bench(Birch(n_clusters=3), "Birch", X_normalized)

print("\nClusters: 4 (non-normalized data):")
bench(Birch(n_clusters=4), "Birch", X)
print("\nClusters: 4 (normalized data):")
bench(Birch(n_clusters=4), "Birch", X_normalized)

#PCA
print("\nData with PCA:")
print("\nClusters: 2 (PCA non-normalized data):")
bench(Birch(n_clusters=2), "Birch", x_pca)
print("\nClusters: 2 (PCA normalized data):")
bench(Birch(n_clusters=2), "Birch", X_pca_normalized)

print("\nClusters: 3 (PCA non-normalized data):")
bench(Birch(n_clusters=3), "Birch", x_pca)
print("\nClusters: 3 (PCA normalized data):")
bench(Birch(n_clusters=3), "Birch", X_pca_normalized)

print("\nClusters: 4 (PCA non-normalized data):")
bench(Birch(n_clusters=4), "Birch", x_pca)
print("\nClusters: 4 (PCA normalized data):")
bench(Birch(n_clusters=4), "Birch", X_pca_normalized)


Clusters: 2 (non-normalized data):
Name		Clusters	Time	Silhouette
Birch    		2	0.0180s	0.659

Clusters: 2 (normalized data):
Name		Clusters	Time	Silhouette
Birch    		2	0.0175s	0.236

Clusters: 3 (non-normalized data):
Name		Clusters	Time	Silhouette
Birch    		3	0.0179s	0.565

Clusters: 3 (normalized data):
Name		Clusters	Time	Silhouette
Birch    		3	0.0171s	0.250

Clusters: 4 (non-normalized data):
Name		Clusters	Time	Silhouette
Birch    		4	0.0180s	0.561

Clusters: 4 (normalized data):
Name		Clusters	Time	Silhouette
Birch    		4	0.0170s	0.203

Data with PCA:

Clusters: 2 (PCA non-normalized data):
Name		Clusters	Time	Silhouette
Birch    		2	0.0188s	0.659

Clusters: 2 (PCA normalized data):
Name		Clusters	Time	Silhouette
Birch    		2	0.0073s	0.375

Clusters: 3 (PCA non-normalized data):
Name		Clusters	Time	Silhouette
Birch    		3	0.0169s	0.566

Clusters: 3 (PCA normalized dataX_pca_normalized
Name		Clusters	Time	Silhouette
Birch    		3	0.0070s	0.460

Clusters: 4 (PCA non-normalized d

## **Conclusões**


> **Silhueta** A médida silhueta nos proporciona uma ideia de como os clusters foram distribuídos. É calculada a partir da média das distâncias intra-cluster e a distância média do cluster mais próximo, para cada amostra. O melhor valor para a silhueta é 1, que indica clusters bem distribuídos, sem sobreposição. Silhueta próxima de 0 indica sobreposição entre clusters. Silhueta negativa indica erro de agrupamento.

**Melhores resultados**

Os melhores resultados foram obtidos a partir da base não normalizada. Deste modo, os três métodos de agrupamento testados convergiram, com um número de clusters = 2, para uma silhueta próxima de 0.65, o que indica uma boa distribuição de clusters. A redução de atributos não surtiu efeitos consideráveis, sejam positivos ou negativos.

O melhor algoritmo, em tempo de processamento, foi o método Aglomerativo (hierárquico), com uma ótima margem em relação aos demais testados. 

**Normalização da base**

Os testes com a base de dados normalizada apresentaram resultados ruins, em comparação aos feitos com base não normalizada, para todos os casos. O melhor resultado de agrupamento com a base normalizada foi alcançado após a redução de atributos com o método PCA, com um valor de silhueta próximo de 0.40. Todos os três métodos testados convergiram para este resultado.

### **Considerações finais**

O melhor algoritmo testado foi o método Aglomerativo (hierárquico). Com ele, foram alcançados bons resultados de silhueta, equivalentes aos métodos K-means e Birch, com uma boa vantagem de tempo de processamento em relação aos demais.